In [1]:
import numpy as np
import scipy as scp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import mpmath
from mpmath import mp

from numba import jit
from numba import cuda

import os
import glob
import time
import IPython.display

import itertools

from tqdm.notebook import tqdm

from lat_class_parallel_map import *
"""
from tqdm.notebook import tqdm
"""

'\nfrom tqdm.notebook import tqdm\n'

In [2]:
"""determine weights"""

mp.prec=100

eta = 4.01
lam = 1.000
mu = 1.050

n_min = 0
n_max = 202
"""
num_W_val = (n_max - n_min)//2

#W = []
#W_obs_ratio = []

W = np.zeros(num_W_val)
W_obs_ratio = np.zeros(num_W_val)

def weight_integrand(r, n, e, l):
    return mpmath.power(r, (n+1)) * mpmath.exp(- e * r ** 2 - l * r ** 4)

for n in tqdm(range(n_min, n_max, 2)):
    weight_integral = mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])
    #W.append(weight_integral)
    W[n//2] = weight_integral
    obs_ratio_integral =  mpmath.quad(lambda x:weight_integrand(x,n+2,eta,lam), [0, mpmath.inf])/mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])
    #W_obs_ratio.append(obs_ratio_integral)
    W_obs_ratio[n//2] = obs_ratio_integral
"""

'\nnum_W_val = (n_max - n_min)//2\n\n#W = []\n#W_obs_ratio = []\n\nW = np.zeros(num_W_val)\nW_obs_ratio = np.zeros(num_W_val)\n\ndef weight_integrand(r, n, e, l):\n    return mpmath.power(r, (n+1)) * mpmath.exp(- e * r ** 2 - l * r ** 4)\n\nfor n in tqdm(range(n_min, n_max, 2)):\n    weight_integral = mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])\n    #W.append(weight_integral)\n    W[n//2] = weight_integral\n    obs_ratio_integral =  mpmath.quad(lambda x:weight_integrand(x,n+2,eta,lam), [0, mpmath.inf])/mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])\n    #W_obs_ratio.append(obs_ratio_integral)\n    W_obs_ratio[n//2] = obs_ratio_integral\n'

In [3]:
W_obs_ratio = np.loadtxt("W_obs_ratio.txt")
W = np.loadtxt("W.txt")

In [4]:
kl_sim = lattice_sim()
kl_sim.mu = mu

[2 2 3 3]
Setting worm to tail: [[0 1]], head: [[[0 1]
  [0 1]]]

with k_delta: [1]

white sites: 5
black sites: 4


In [5]:
W

array([1.12933110e-01, 2.35691145e-02, 9.21048041e-03, 5.10210128e-03,
       3.58600756e-03, 3.01425739e-03, 2.92143284e-03, 3.18529932e-03,
       3.83848979e-03, 5.04502526e-03, 7.15792842e-03, 1.08734798e-02,
       1.75672793e-02, 3.00184838e-02, 5.40002555e-02, 1.01858875e-01,
       2.00774872e-01, 4.12317378e-01, 8.79890073e-01, 1.94667680e+00,
       4.45586870e+00, 1.05327513e+01, 2.56684551e+01, 6.43950117e+01,
       1.66075235e+02, 4.39759295e+02, 1.19422305e+03, 3.32245362e+03,
       9.46049163e+03, 2.75460650e+04, 8.19472683e+04, 2.48886702e+05,
       7.71164821e+05, 2.43600177e+06, 7.84003602e+06, 2.56927578e+07,
       8.56866509e+07, 2.90667905e+08, 1.00241389e+09, 3.51285035e+09,
       1.25038059e+10, 4.51868761e+10, 1.65728335e+11, 6.16639085e+11,
       2.32679784e+12, 8.90083021e+12, 3.45067868e+13, 1.35532987e+14,
       5.39165850e+14, 2.17176417e+15, 8.85517618e+15, 3.65394759e+16,
       1.52545344e+17, 6.44172959e+17, 2.75088482e+18, 1.18771458e+19,
      

In [6]:
#seed = 42
#np.random.seed(seed)

In [ ]:
phi2_obs_file = open("phi2_obs.txt","w")
n_obs_file = open("n_obs.txt","w")

num_iter = 50

iter_per_obs = 5

worms_per_iter = 1

mcmc_per_iter = 1

phi2s = np.zeros(shape=(num_iter//iter_per_obs))
ns = np.zeros(shape=(num_iter//iter_per_obs))

i_obs = 0

print("starting worm k algorithm")

for i in tqdm(range(num_iter)):
    
    ##worm algorithm k sampling
    ##count the number of worms for one iterations
    worm_count = 0
    
    
    ##limit the number of worms in one iteration
    while worm_count < worms_per_iter:
        
        print("starting worm")
        kl_sim.reset_worm_i_worm(0,
                               kl_sim.k_links, kl_sim.l_links, kl_sim.mu, kl_sim.W, kl_sim.f,
                               kl_sim.old_head_lat, kl_sim.work_head_lat, kl_sim.tail_df_lat,
                               kl_sim.move_dim, kl_sim.move_sign, kl_sim.move_i, kl_sim.move_set, kl_sim.moves,
                               kl_sim.head, kl_sim.tail, kl_sim.worm_link_coord, kl_sim.worm_closed,
                               kl_sim.k_delta, kl_sim.dk, kl_sim.df, kl_sim.df_prev_iter, kl_sim.df0,
                               kl_sim.p_acc,
                               kl_sim.num_worms, kl_sim.lat_size, kl_sim.dim
                                )
        
        print("started worm")
        ##let the worm run AND close in this loop
        
        kl_sim.run_i_worm(0,
                       kl_sim.k_links, kl_sim.l_links, kl_sim.mu, kl_sim.W, kl_sim.f,
                       kl_sim.old_head_lat, kl_sim.work_head_lat, kl_sim.tail_df_lat,
                       kl_sim.move_dim, kl_sim.move_sign, kl_sim.move_i, kl_sim.move_set, kl_sim.moves,
                       kl_sim.head, kl_sim.tail, kl_sim.worm_link_coord, kl_sim.worm_closed,
                       kl_sim.k_delta, kl_sim.dk, kl_sim.df, kl_sim.df_prev_iter, kl_sim.df0,
                       kl_sim.p_acc,
                       kl_sim.num_worms, kl_sim.lat_size, kl_sim.dim
    
                         )
        
        
        print("worm has closed")
        worm_count += 1
        
    print("ended worm k algorithm")
    ##mcmc l sweep
    print("f:")
    print(kl_sim.f)
    print("\n")
    print("starting mcmc l sweep")
    l_sweep = 0
    while l_sweep < mcmc_per_iter:
        
        ##sample white sites
        kl_sim.sweep(
              kl_sim.w_sites, kl_sim.w_target_sites, kl_sim.n_w_sites,
              kl_sim.k_links, kl_sim.l_links, kl_sim.W, kl_sim.f,
              kl_sim.w_link_coord, kl_sim.mcmc_w_mod_dim, kl_sim.mcmc_w_dl, kl_sim.mcmc_w_df, kl_sim.mcmc_w_target_df,
              kl_sim.move_set, kl_sim.w_moves,
              kl_sim.mcmc_w_p_acc,
              kl_sim.lat_size, kl_sim.dim
             )

        ##sample black sites
        kl_sim.sweep(
              kl_sim.b_sites, kl_sim.b_target_sites, kl_sim.n_b_sites,
              kl_sim.k_links, kl_sim.l_links, kl_sim.W, kl_sim.f,
              kl_sim.b_link_coord, kl_sim.mcmc_b_mod_dim, kl_sim.mcmc_b_dl, kl_sim.mcmc_b_df, kl_sim.mcmc_b_target_df,
              kl_sim.move_set, kl_sim.b_moves,
              kl_sim.mcmc_b_p_acc,
              kl_sim.lat_size, kl_sim.dim
             )
        
        l_sweep += 1
        
            
    #print(f"f: {kl_sim.f}")
    print("ended mcmc l sweep")
    
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)
    
    if i%iter_per_obs == 0:
        
        ##update observables
        ##phi2
        phi2 = 0.
        ##n
        n = 0.
        
        for coord in itertools.product(kl_sim.lat_size):
            phi2 += W_obs_ratio[kl_sim.f[tuple(coord)]]
            n += kl_sim.k_links[tuple(np.concatenate([0,coord]))]
            
        phi2 = phi2/kl_sim.num_sites
        n = n/kl_sim.num_sites
        
        #phi2_obs_file.write(phi2)
        #n_obs_file.write(n)
        
        phi2s[i_obs] = phi2
        n2s[i_obs] = n2
        
        i_obs += 1

starting worm k algorithm


starting worm
resetting worm
f before:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
f after:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
proposing new heads:
 [[0 1]]
dk: [1]
df: [1]
f_old 0
df[i_worm] 1
f_prop_head 0
l_old_link 0
k_proposed_link 1
k_old_link 0
saving df_prev_iter[i_worm] 1
Resetting bits at prop_head [0 1]=False and old_head [2 1]=False
worm closed: True
worm closed: False
Resetting worm 0 to tail: [2 1], head: [0 1]

with k_delta: 1

started worm
proposing new heads:
 [[0 2]]
dk: [1]
df: [1]
f_old 0
df[i_worm] 1
f_prop_head 0
df_prop_head 0
l_old_link 0
k_proposed_link 1
k_old_link 0
p_acc: 0.20869977375671017
p_draw: 0.2754903884015574
accepted: 0
f before:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
old head (0, 1)
df[i_worm] 1
df_prev_iter[i_worm] 1
f after:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
saving df_prev_iter[i_worm] 1
Resetting bits at prop_head [0 2]=False and old_head [0 1]=False
head: [0 1], tail: [2 1]
worm closed: False
running l_pos_check
l >= 0: True 
running f_check
calculated f:
[[0. 1. 0.]
 [0. 0. 0.]
 [0

In [ ]:
for site_coords in kl_sim.all_site_coords:
    print(site_coords)

In [ ]:
"""
What is missing?
Save df0 when starting worms

correct df update worm algorithm

correct p_acc?

replace if statements with expressions
(per_lat_coord)

refrain from exclusive python functionality as much as possible

(combine i_worm loops for more efficiency and parallelization)

Set df correctly
Check for tails and accomodate for df0

"""
